In [10]:
import pandas as pd
from docx import Document, types
import docx
import unicodedata
import re
import os
import glob
# doc = Document("/Users/kong/Documents/เอกสารประกาศ KB/25.ตรวจคัดกรองมะเร็งปากมดลูก/ตรวจคัดกรองมะเร็งปากมดลูก.docx")
def read_docx_table(document, table_num=1, nheader=1):
    table = document.tables[table_num-1]
    data = [[re.sub(r'\s+',' ',unicodedata.normalize("NFKD",cell.text).replace('\n','').replace('\t','').strip()) for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    if nheader == 1:
        df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    elif nheader == 2:
        outside_col, inside_col = df.iloc[0], df.iloc[1]
        hier_index = pd.MultiIndex.from_tuples(list(zip(outside_col, inside_col)))
        df = pd.DataFrame(data, columns=hier_index).drop(df.index[[0,1]]).reset_index(drop=True)
    elif nheader > 2:
        raise Exception("More than two headers not currently supported")
    return df

def read_p_tb(path):
    doc = Document(path)
    tb_count = 1
    passages = ['']
    passage = ''
    for idx , element in enumerate(doc.element.body):
        id = len(passages)
        if id == 0:
            id = 0
        else:
            id -= 1
        # print(element.tag)
        if element.tag.endswith('p'):  # Check if the element is a paragraph
            paragraph = element
            line = paragraph.text
            line = unicodedata.normalize("NFKD",line).replace('\n','').replace('\t','').strip()
            line = re.sub(r'\s+',' ',line)
            if line not in ['',' ']:
                if len(passage)+len(line)<8001:
                    passage+=' '+line
                    passages[id] = passage
                else:
                    passage = line
                    passages.append(passage)
                # print(line)  # Print or process the paragraph text
        elif element.tag.endswith('tbl'):  # Check if the element is a table
            tb = read_docx_table(doc, table_num=tb_count, nheader=1)
            tb_dict = f"ตารางมีคอลัมน์ {list(tb.columns)}: {tb.to_dict('records')}"
            if len(passage)+len(tb_dict)<8001:
                passage+=' '+tb_dict
                passages[id] = passage
            else:
                passage = tb_dict
                passages.append(passage)
            # print(tb_dict)
            tb_count+=1

        elif element.tag.endswith('sectPr'):
            pass

    return passages

def upadate_new_docx(path,data_path):
    data = pd.read_csv(data_path)
    passages = read_p_tb(path)
    file_name = [os.path.basename(path).split('.')[0]] * len(passages)
    file_path = [path]* len(passages)
    for i in range(len(passages)):
        new_data = {'File_name':file_name[i],'File_path':file_path[i],'Passage':passages[i]}
        data = data._append(new_data, ignore_index = True)
    # print(data['Passage'])
    data.to_csv(data_path,index=False)
    print('update success')

# read_docx_table(doc).to_dict('records')

In [11]:
path = "/Users/kong/Documents/mod/RAG_CC67/folder_rag/ตรวจคัดกรองมะเร็งปากมดลูก.docx"
passage = read_p_tb(path)
passage

[' ตรวจคัดกรองมะเร็งปากมดลูก กลุ่มเป้าหมาย : หญิงที่มีสัญชาติไทย มีเลขบัตรประจําตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว) 1. อายุ 30 - 59 ปี ทุกคน 2. อายุ 15-29 ปีที่มีความเสี่ยงสูง ทั้งนี้ขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย หมายเหตุ : สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 ต้องการให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจํานวนมาก ให้1330 รับเรื่องเข้ามาก่อน เปิดประเด็นบริการ--->การตรวจคัดกรองมะเร็งปากมดลูก ---โอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้ //กรณีพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จํานวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย (อัปเดต 15/1/67 เวลา 14.00 น.) ข่าวเริ่มแคมเป

In [274]:
passage[0]

" บริการฟื้นฟูสมรรถภาพด้านการแพทย์ 1. คนพิการได้รับอุปกรณ์เครื่องช่วยคนพิการตามความจําเป็นอย่างทั่วถึงและครอบคลุม 2. คนพิการ ผู้สูงอายุที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์และผู้ป่วยที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ ได้รับบริการด้านการฟื้นฟูสมรรถภาพด้านการแพทย์อย่างต่อเนื่องทั้งในหน่วยบริการและในชุมชน 3. ส่งเสริมให้เกิดความร่วมมือระหว่างหน่วยบริการ องค์กรปกครองส่วนท้องถิ่น องค์กรคนพิการองค์กรภาคีที่เกี่ยวข้อง และชุ มชน ในการพัฒนารูปแบบการดูแล ช่วยเหลือ สนับสนุน และให้บริการฟื้นฟูสมรรถภาพแก่กลุ่มเป้าหมายร่วมกัน ทั้งในหน่วยบริการและในชุมชนอย่างยั่งยืน 1. กลุ่มเป้าหมาย ครอบคลุมผู้มีสิทธิหลักประกันสุขภาพแห่งชาติ 3 กลุ่ม ดังนี้ 1. คนพิการ (รหัสสิทธิย่อย 74) 2. ผู้สูงอายุที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ 3. ผู้ป่วยที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ 2. สิทธิประโยชน์ที่ได้รับ ครอบคลุมบริการ ค่าอุปกรณ์เครื่องช่วยสําหรับคนพิการ ค่าบริการฟื้นฟูสมรรถภาพด้านการแพทย์ระยะกลาง (Intermediate care :IMC) การฝึกทักษะการดํารงชีวิตอิสระสําหรับคนพิกา

In [12]:
data_path = '/Users/kong/Documents/mod/RAG_CC67/folder_rag/Data.csv'
# path = "file/folder_rag/"
# for i in glob.glob(path + '*.docx'):
#     # print(i)
#     upadate_new_docx(i,data_path)

In [13]:
data = pd.read_csv(data_path)
data

File_name  \
0               ตรวจคัดกรองมะเร็งปากมดลูก   
1                  การใช้สิทธิย่อยคนพิการ   
2  สิทธิไม่ตรงตามจริง ติดสิทธิประกันสังคม   
3        บริการฟื้นฟูสมรรถภาพด้านการแพทย์   
4                         การใช้สิทธิว่าง   

                                           File_path  \
0     file/folder_rag/ตรวจคัดกรองมะเร็งปากมดลูก.docx   
1        file/folder_rag/การใช้สิทธิย่อยคนพิการ.docx   
2  file/folder_rag/สิทธิไม่ตรงตามจริง ติดสิทธิประ...   
3  file/folder_rag/บริการฟื้นฟูสมรรถภาพด้านการแพท...   
4               file/folder_rag/การใช้สิทธิว่าง.docx   

                                             Passage  
0   ตรวจคัดกรองมะเร็งปากมดลูก กลุ่มเป้าหมาย : หญิ...  
1   การใช้สิทธิย่อยคนพิการ (ท74) 1.กลุ่มเป้าหมาย ...  
2   ติดสิทธิประกันสังคม (สิทธิไม่ตรงตามจริง) กลุ่...  
3   บริการฟื้นฟูสมรรถภาพด้านการแพทย์ 1. คนพิการได...  
4   การใช้สิทธิว่าง (มาตรา8 ตามพ.ร.บ.หลักประกันสุ...

In [14]:
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
# from tqdm import tqdm
from tqdm.auto import tqdm  # for notebooks
import string
# from FlagEmbedding import BGEM3FlagModel
import numpy as np
tqdm.pandas()
os.environ["OPENTYPHOON_API_KEY"] = "sk-PLQ1EWaCPQZc0guiwHdCQMYUe3GsTZGFEdg3uLI2qcRKz4mA"
client = OpenAI(api_key=os.environ["OPENTYPHOON_API_KEY"], base_url="https://api.opentyphoon.ai/v1")

/Users/kong/Documents/mod/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
yn_prompt = '''
You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Query: How to plant a tree?
Document: """Cars were invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.[3][4][5] Cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages.[6] In Europe and other parts of the world, demand for automobiles did not increase until after World War II.[7] The car is considered an essential part of the developed economy."""
Relevant: No

Query: Has the coronavirus vaccine been approved?
Document: """The Pfizer-BioNTech COVID-19 vaccine was approved for emergency use in the United States on December 11, 2020."""
Relevant: Yes

Query: What is the capital of France?
Document: """Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Beyond such landmarks as the Eiffel Tower and the 12th-century, Gothic Notre-Dame cathedral, the city is known for its cafe culture and designer boutiques along the Rue du Faubourg Saint-Honoré."""
Relevant: Yes

Query: What are some papers to learn about PPO reinforcement learning?
Document: """Proximal Policy Optimization and its Dynamic Version for Sequence Generation: In sequence generation task, many works use policy gradient for model optimization to tackle the intractable backpropagation issue when maximizing the non-differentiable evaluation metrics or fooling the discriminator in adversarial learning. In this paper, we replace policy gradient with proximal policy optimization (PPO), which is a proved more efficient reinforcement learning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We demonstrate the efficacy of PPO and PPO-dynamic on conditional sequence generation tasks including synthetic experiment and chit-chat chatbot. The results show that PPO and PPO-dynamic can beat policy gradient by stability and performance."""
Relevant: Yes

Query: Explain sentence embeddings
Document: """Inside the bubble: exploring the environments of reionisation-era Lyman-α emitting galaxies with JADES and FRESCO: We present a study of the environments of 16 Lyman-α emitting galaxies (LAEs) in the reionisation era (5.8<z<8) identified by JWST/NIRSpec as part of the JWST Advanced Deep Extragalactic Survey (JADES). Unless situated in sufficiently (re)ionised regions, Lyman-α emission from these galaxies would be strongly absorbed by neutral gas in the intergalactic medium (IGM). We conservatively estimate sizes of the ionised regions required to reconcile the relatively low Lyman-α velocity offsets (ΔvLyα<300kms−1) with moderately high Lyman-α escape fractions (fesc,Lyα>5%) observed in our sample of LAEs, indicating the presence of ionised ``bubbles'' with physical sizes of the order of 0.1pMpc≲Rion≲1pMpc in a patchy reionisation scenario where the bubbles are embedded in a fully neutral IGM. Around half of the LAEs in our sample are found to coincide with large-scale galaxy overdensities seen in FRESCO at z∼5.8-5.9 and z∼7.3, suggesting Lyman-α transmission is strongly enhanced in such overdense regions, and underlining the importance of LAEs as tracers of the first large-scale ionised bubbles. Considering only spectroscopically confirmed galaxies, we find our sample of UV-faint LAEs (MUV≳−20mag) and their direct neighbours are generally not able to produce the required ionised regions based on the Lyman-α transmission properties, suggesting lower-luminosity sources likely play an important role in carving out these bubbles. These observations demonstrate the combined power of JWST multi-object and slitless spectroscopy in acquiring a unique view of the early stages of Cosmic Reionisation via the most distant LAEs."""
Relevant: No

Query: {query}
Document: """{document}"""
Relevant:
'''

In [16]:
list_document ='''TITLE: {title} 
DOCUMENT: """"{document}"""'''

ans_prompt ='''
You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:

Keep your answer ground in the facts provided in DOCUMENT section.
If the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].

{template}

QUESTION: {question}
ANSWER:
'''

In [17]:
extract_prompt = """QUESTION
{query}

DOCUMENT
{document}

INSTRUCTIONS
Read the QUESTION carefully.
Examine the DOCUMENT section for any passages that provide the facts required to answer the QUESTION.
If the DOCUMENT contains the necessary information, extract and return the relevant passage.
If the DOCUMENT does not contain the necessary information, return [NONE].
Your extracted passage or [NONE] must directly address the QUESTION.
OUTPUT

"""

In [18]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def retrival_70b(query,document):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": yn_prompt.format(query=query, document=document)}],
        max_tokens=1,
        temperature= 0,
        top_p = 0.9
    )

    return response.choices[0].message.content

# @retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def extract_passage(query,document):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": extract_prompt.format(query=query, document=document)}],
        # max_tokens=1,
        temperature= 0.3,
        top_p = 0.9
    )

    return response.choices[0].message.content

def clean_yn(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.translate(str.maketrans('', '', string.digits))
    return text.strip()

def answer(question,template):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": ans_prompt.format(question=question, template=template)}],
        max_tokens=4096,
        temperature= 0.3,
        top_p = 0.9
    )

    return response.choices[0].message.content

def chat_rag(query,docs):
    global temp
    f_yn = []
    for i in docs['Passage']:
        f_yn.append(clean_yn(retrival_70b(query,i)))
    if 'yes' in f_yn:
        idx = [i for i,x in enumerate(f_yn) if x=='yes']
    else:
        return 'None'
    print(idx)
    contents = []
    for content in docs['Passage'][idx]:
        print(content)
        Ans = extract_passage(query,content)
        contents.append(Ans)
        print(Ans)
    template = '\n'.join([list_document.format(title=i, document = j) for i,j in zip(docs['File_name'][idx],contents)])
    ans = answer(query,template)
    if clean_yn(ans) == 'none':
        return ans
    else: 
        return (ans,idx)
    

In [19]:
tb = chat_rag('pep smear คืออะไร',data)
tb

[0]
 ตรวจคัดกรองมะเร็งปากมดลูก กลุ่มเป้าหมาย : หญิงที่มีสัญชาติไทย มีเลขบัตรประจําตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว) 1. อายุ 30 - 59 ปี ทุกคน 2. อายุ 15-29 ปีที่มีความเสี่ยงสูง ทั้งนี้ขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย หมายเหตุ : สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 ต้องการให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจํานวนมาก ให้1330 รับเรื่องเข้ามาก่อน เปิดประเด็นบริการ--->การตรวจคัดกรองมะเร็งปากมดลูก ---โอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้ //กรณีพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จํานวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย (อัปเดต 15/1/67 เวลา 14.00 น.) ข่าวเริ่มแคม

('Pap smear คือ วิธีการตรวจคัดกรองหาระยะก่อนเป็นมะเร็งปากมดลูก โดยแพทย์จะใช้เครื่องมือสอดผ่านและถ่างช่องคลอด จากนั้นทำการป้ายเซลล์จากมดลูกส่งตรวจทางห้องปฏิบัติการเพื่อตรวจหาเซลล์ที่ผิดปกติ',
 [0])